In [54]:
import numpy as np
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

import warnings
warnings.filterwarnings("ignore")

In [55]:
raw_data_np = np.genfromtxt("loan-data.csv", delimiter = ';', skip_header = 1, autostrip = True, encoding = 'latin9')
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Missing Values

In [56]:
print('This dataset has : ' + str(np.isnan(raw_data_np).sum()) + ' missing values')

This dataset has : 88005 missing values


In [57]:
tmp_fill = np.nanmax(raw_data_np) + 1
tmp_stats = np.array([np.nanmin(raw_data_np, axis = 0),# Minimum value of each column
                           np.nanmean(raw_data_np, axis = 0),# Mean value of each column
                           np.nanmax(raw_data_np, axis = 0)])# Maximum value of each column

When handling missing values in loan data, it is better always to assume the worst because some people withhold information that reduces their chance of getting a loan. Therefore, I will assume the worst and do the following:

- Fill the following columns with a **maximum value**: "Loaned Amount," "Interest Rate," "Total Payment," and "Installment" (according to the maximum value of each column).
- Fill the "Funded Amount" column with its **minimum value**.

## Dividing the Dataset into Numeric and String Data

In [58]:
# Creating an array that contains only indexes of string columns
columns_strings = np.argwhere(np.isnan(tmp_stats[1])).squeeze()

# Creating an array that contains only indexes of numeric columns
columns_numeric = np.argwhere(np.isnan(tmp_stats[1]) == False).squeeze()

In [59]:
# Importing only the string data
loan_data_strings = np.genfromtxt("loan-data.csv",
                                  delimiter = ';',
                                  skip_header = 1,
                                  autostrip = True, 
                                  usecols = columns_strings,
                                  dtype = str,
                                  encoding = 'latin9')
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [60]:
# Importing only the numeric data 
loan_data_numeric = np.genfromtxt("loan-data.csv",
                                  delimiter = ';',
                                  autostrip = True,
                                  skip_header = 1,
                                  usecols = columns_numeric,
                                  encoding = 'latin9',
                                  filling_values = tmp_fill) # Filling all NaN values with the tmp_fill value  
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

**----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**


In [61]:
# Importing only the headers
header = np.genfromtxt("loan-data.csv",
                            delimiter = ';',
                            autostrip = True,
                            skip_footer = raw_data_np.shape[0],
                            dtype = np.str,
                           encoding = 'latin9')
header_strings, header_numeric = header[columns_strings], header[columns_numeric]

In [62]:
print(f'Headers of Numeric Data: {header_numeric}')
print(f'Headers of String Data: {header_strings}')

Headers of Numeric Data: ['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']
Headers of String Data: ['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']


# Numeric Data

In [11]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

## Replacing "Temporary Filled" Values.
I will replace the **temporarily filled values** in each numeric column in the following way:

- Fill the following columns with a **maximum value**: "Loaned Amount," "Interest Rate," "Total Payment," and "Installment" (according to the maximum value of each column).
- Fill the "Funded Amount" column with its **minimum value**.

In [12]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

### ID

In [14]:
np.isin(loan_data_numeric[:,0], tmp_fill).sum() 
# The column ID does not have any NaN values as it does not contain the tmp_fill value.

0

### Loaned Amount, Interest Rate, Total Payment, Installment

In [15]:
""" 
Numeric index of each column:
    loan_amnt:1,
    int_rate:3,
    installment:4,
    total_pymnt: 5
"""
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [16]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [17]:
for i in [1,3,4,5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == tmp_fill,
                                      tmp_stats[2, columns_numeric[i]],
                                      loan_data_numeric[:,i])
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

### Funded Amount

In [18]:
header_numeric 

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [19]:

loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == tmp_fill, 
                                  tmp_stats[0, columns_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

# String Data

In [20]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [21]:
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

### Issue Date

In [22]:
print(f"The column with the '{header_strings[0]}' header has the following unique values: \n {np.unique(loan_data_strings[:,0])}")

The column with the 'issue_d' header has the following unique values: 
 ['' 'Apr-15' 'Aug-15' 'Dec-15' 'Feb-15' 'Jan-15' 'Jul-15' 'Jun-15' 'Mar-15' 'May-15' 'Nov-15'
 'Oct-15' 'Sep-15']


In [23]:
# Assigning numerical representation to each month.
months = np.array(['', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15', 'Jul-15', 'Aug-15', 'Sep-15', 'Oct-15', 'Nov-15', 'Dec-15'])
for i in range(13):
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])
        
np.unique(loan_data_strings[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Loan Status

In [ ]:
print(f'The column with the '{header_strings[1]}' header has the following unique values: \n {np.unique(loan_data_strings[:,1])}')

The column with the loan_status header has the following unique values: 
 ['' 'Charged Off' 'Current' 'Default' 'Fully Paid' 'In Grace Period' 'Issued' 'Late (16-30 days)'
 'Late (31-120 days)']


In [24]:
# Dividing loan status into two groups: bad and good loan status. 
status_bad = np.array(['','Charged Off','Default','Late (31-120 days)'])

# Assigning numerical representation to bad and good loan status.
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad),0,1)
np.unique(loan_data_strings[:,1])

array(['0', '1'], dtype='<U69')

### Term

In [25]:
print(f'The column with the '{header_strings[2]}' header has the following unique values: \n {np.unique(loan_data_strings[:,2])}')

The column with the term header has the following unique values: 
 ['' '36 months' '60 months']


In [26]:
# Removing 'months' word
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], " months")
loan_data_strings[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [27]:
# Replacing '' values to 60
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '', 
                                  '60', 
                                  loan_data_strings[:,2])
loan_data_strings[:,2]

array(['36', '36', '36', ..., '36', '36', '36'], dtype='<U69')

In [28]:
np.unique(loan_data_strings[:,2])

array(['36', '60'], dtype='<U69')

### Grade and Subgrade

In [31]:
print(f"The column with the '{header_strings[3]}' header has the following unique values: \n {np.unique(loan_data_strings[:,3])}")

The column with the grade header has the following unique values: 
 ['' 'A' 'B' 'C' 'D' 'E' 'F' 'G']


In [32]:
print(f"The column with the '{header_strings[4]}' header has the following unique values: \n {np.unique(loan_data_strings[:,4])}")

The column with the sub_grade header has the following unique values: 
 ['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


In [33]:
print(f"The column 'Grade' has {np.unique(loan_data_strings[:,3],return_counts = True)[1][0]} empty string ('') values")
print(f"The column 'Subgrade' has {np.unique(loan_data_strings[:,4],return_counts = True)[1][0]} empty string ('') values")


The column 'Gradee' has 515 empty string ('') values
The column 'Subgrade' has 514 empty string ('') values


In [34]:
np.unique(loan_data_strings[:,4],return_counts = True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([514, 285, 278, 239, 323, 502, 509, 517, 530, 553, 494, 629, 567, 586, 564, 423, 391, 267,
        250, 255, 223, 235, 162, 171, 139, 114,  94,  52,  34,  43,  16,  19,  10,   3,   7,   2]))

- The columns Grade and Subgrade convey the same information; therefore, it will be better to remove one of them.
- I have decided to remove the column "Grade" as it contains general information, while the column "Subgrade" includes detailed information about the grades. 
- However, before dropping the column "Grade," I will check if it is possible to fill the empty strings of column 'Subgrade' **(' ')** with the 'Grade' values.

### Filling Subgrade's Empty Strings

In [35]:
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

In [36]:
np.unique(loan_data_strings[:,4], return_counts = True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

In [37]:
loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                  'H1',
                                  loan_data_strings[:,4])

In [38]:
# After filling all empty string ('') values
np.unique(loan_data_strings[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [39]:
# Dropping Grade
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)
loan_data_strings[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

### Assigning numerical representation to each Subgrade

In [40]:
keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))
print(dict_sub_grade)

{'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30, 'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35, 'H1': 36}


In [41]:
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])
np.unique(loan_data_strings[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Verification Status

In [42]:
print(f"The column with the '{header_strings[4]}' header has the following unique values: \n {np.unique(loan_data_strings[:,4])}")

The column with the sub_grade header has the following unique values: 
 ['' 'Not Verified' 'Source Verified' 'Verified']


In [43]:
np.unique(loan_data_strings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [44]:
# Assigning numerical representation to each status of the column: "Verification Status."
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') | (loan_data_strings[:,4] == 'Not Verified'), 0, 1)
np.unique(loan_data_strings[:,4])

array(['0', '1'], dtype='<U69')

### URL

In [45]:
loan_data_strings[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

In [46]:
# Removing the string value from each URL address, leaving only the number that appears at the end of each URL address.
np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [47]:
# Comparing column 'ID' values and 'URL's numeric values
np.array_equal(loan_data_numeric[:,0].astype(dtype = np.int32),loan_data_strings[:,5].astype(dtype = np.int32))

True

- The numeric values from the URL are identical to the values in column 'id.' 
<br>Thus, I will drop the URL column as it conveys the same information of the column 'id.'


In [ ]:
# Dropping column URL
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)

### State Address

In [ ]:
print(f"The column with the '{header_strings[5]}' header has the following unique values: \n {np.unique(loan_data_strings[:,5])}")

The column with the verification_status header has the following unique values: 
 ['' 'AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD'
 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI'
 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


In [ ]:
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10]))

In [ ]:
# Assigning zero values to empty string values

loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '', 
                                  0, 
                                  loan_data_strings[:,5])

In [ ]:
# Grouping states by regions

states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [ ]:
# Assigning numerical representation to each region

loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [ ]:
np.unique(loan_data_strings[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

## Converting the dtype of String Data to Integer & Removing the Dropped Columns' Headers.

In [ ]:
# Removing the "Grade" and "URL" headers
# "Grade" index: 3  
# "URL" index: 6

header_strings = np.delete(header_strings,[3,6]) 

In [ ]:
print(f" Data type of the string columns before changing  to 'int':{loan_data_strings.dtype} ")
for i in range(5):
    print(f"{header_strings[i]} : {loan_data_strings[:i].dtype}")

 Data type of the string columns before changing  to 'int':<U69 
issue_d : <U69
loan_status : <U69
term : <U69
sub_grade : <U69
verification_status : <U69


In [ ]:
# Converting all str dtype values to int.
loan_data_strings = loan_data_strings.astype(np.int)
loan_data_strings

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

## Stucking the Numeric, String Data and Headers

In [ ]:
pre_processed_data = np.hstack((loan_data_numeric , loan_data_strings))
pre_processed_data.shape

(10000, 12)

In [ ]:
print(f'Shape of numeric data: {loan_data_numeric.shape}')
print(f'Shape of string data: {loan_data_strings.shape}')
print(f'Shape of dataset after stucking numeric and string data: {pre_processed_data.shape}')

Shape of numeric data: (10000, 6)
Shape of string data: (10000, 6)
Shape of dataset after stucking numeric and string data: (10000, 12)


In [ ]:
final_header = np.concatenate((header_numeric, header_strings))

In [ ]:
# Sorting Dataset by id column
pre_processed_data = pre_processed_data[np.argsort(pre_processed_data[:,0])]
pre_processed_data

array([[  373332.,     9950.,     1000., ...,       21.,        0.,        1.],
       [  575239.,    12000.,    12000., ...,       25.,        1.,        2.],
       [  707689.,    10000.,    10000., ...,       13.,        1.,        0.],
       ...,
       [68614880.,     5600.,     5600., ...,        8.,        1.,        1.],
       [68615915.,     4000.,     4000., ...,       10.,        1.,        2.],
       [68616519.,    21600.,    21600., ...,        3.,        0.,        2.]])

## Saving the Pre-processed Data

In [ ]:
pre_processed_data = np.vstack((final_header, pre_processed_data))

In [ ]:
np.savetxt("pre_processed_loan-data.csv", 
           pre_processed_data, 
           fmt = '%s',
           delimiter = ',')